<a href="https://colab.research.google.com/github/AFalabella0/SpeechBrain/blob/master/prova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install speechbrain

In [8]:
!sudo apt-get install p7zip-full
!7za x /content/drive/MyDrive/Dati/LibriSpeech.7z

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Dati/                                      1 file, 685409744 bytes (654 MiB)

Extracting archive: /content/drive/MyDrive/Dati/LibriSpeech.7z
--
Path = /content/drive/MyDrive/Dati/LibriSpeech.7z
Type = 7z
Physical Size = 685409744
Headers Size = 44523
Method = LZMA2:24
Solid = +
Blocks = 1

  0%

In [83]:
import speechbrain as sb
import torch
import torchaudio
import os
import pandas as pd
import librosa
import sklearn

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys

import matplotlib.pyplot as plt
import IPython.display as ipd

from tqdm import tqdm

In [46]:
torch.cuda.is_available()
torch.cuda.current_device()
torch.cuda.device(0)
torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [47]:
currentPath = os.getcwd()
path = currentPath + "/LibriSpeech-dev/dev-clean"
df = pd.DataFrame()
for speaker in os.listdir(path):
    path1 = path + "/" + speaker
    for book in os.listdir(path1):
        path2 = path1 + "/" + book
        for name in  os.listdir(path2):
            if "trans" not in name:
                path3 = path2 + "/" + name;
                #print(path3)
                sId = name.split("-")[0]
                wave, sr = librosa.load(path3, duration = 1)
                
                df = df.append([[sId, wave]], ignore_index = True)
df.columns = ["speaker", "wave"]

In [48]:
df["speaker"]

0       6295
1       6295
2       6295
3       6295
4       6295
        ... 
2698    7850
2699    7850
2700    7850
2701    7850
2702    7850
Name: speaker, Length: 2703, dtype: object

torchaudio datasets


In [75]:
libri = torchaudio.datasets.LIBRISPEECH(currentPath, url= "dev-clean", folder_in_archive = 'LibriSpeech', download = True)

In [81]:
data_loader = torch.utils.data.DataLoader(libri, batch_size=36, shuffle=True, num_workers=1)

In [84]:
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=2)


model = M5(n_input=transformed.shape[0], n_output=len(labels))
model.to(device)
print(model)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


n = count_parameters(model)
print("Number of parameters: %s" % n)

NameError: ignored

dataframe

In [79]:
print(libri)

In [35]:
df.head
train_x, train_y, val_x, val_y = sklearn.model_selection.train_test_split(df["wave"], df["speaker"], test_size=0.30, random_state=42)

In [54]:
train, val =  sklearn.model_selection.train_test_split(df, test_size=0.30, random_state=42)

NameError: ignored

In [51]:
train_data = sb.dataio.dataloader.SaveableDataLoader(train)
val_data = sb.dataio.dataloader.SaveableDataLoader(val)

In [52]:
class SimpleBrain(sb.Brain):
  def compute_forward(self, batch, stage):
    return self.modules.model(batch["input"])

    
  def compute_objectives(self, predictions, batch, stage):
    return torch.nn.functional.l1_loss(predictions, batch["target"])

model = torch.nn.Linear(in_features=10, out_features=10)
brain = SimpleBrain({"model": model}, opt_class=lambda x: torch.optim.SGD(x, 0.1))
data = [{"input": torch.rand(10, 10), "target": torch.rand(10, 10)}]
brain.fit(epoch_counter = range(10), train_set = train_data, valid_set = val_data, progressbar=True)

  0%|          | 0/1892 [00:00<?, ?it/s]


KeyError: ignored

In [80]:
class SimpleBrain(sb.Brain):
  def compute_forward(self, batch, stage):
    return self.modules.model(batch["input"])

    
  def compute_objectives(self, predictions, batch, stage):
    return torch.nn.functional.l1_loss(predictions, batch["target"])

model = torch.nn.Linear(in_features=10, out_features=10)
brain = SimpleBrain({"model": model}, opt_class=lambda x: torch.optim.SGD(x, 0.1))
data = data_loader
brain.fit(range(10), data)

  0%|          | 0/76 [00:00<?, ?it/s]


RuntimeError: ignored